### Utterances por frase con pylangacq

https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f

In [1]:
import pylangacq as pla
import pandas as pd
import spacy
from sklearn.utils import shuffle

In [55]:
eve = pla.read_chat('./Corpus/Pitt/Control/cookie/*.cha', encoding = 'utf-8')
tagged = eve.tagged_sents(participant='PAR')
sentences = eve.sents(participant='PAR')
group = list(['control']*len(tagged))
control_df = pd.DataFrame(
    {'utterances': sentences,
     'tagged': tagged,
     'group': group
    })

In [56]:
eve = pla.read_chat('./Corpus/Pitt/Dementia/cookie/*.cha', encoding = 'utf-8')
tagged = eve.tagged_sents(participant='PAR')
sentences = eve.sents(participant='PAR')
group = list(['dementia']*len(tagged))
dementia_df = pd.DataFrame(
    {'utterances': sentences,
     'tagged': tagged,
     'group': group
    })
#dementia_df.head()

In [57]:
# Unimos los dos df , reorganizamos random y pasamos la colummna utterances de list -> str
df = pd.concat([control_df, dementia_df])
df = shuffle(df).reset_index(drop=True)

utterances_str = []
for i in range(len(df['utterances'])):
    list1 = df['utterances'][i]
    str1 = ' '.join(list1)
    utterances_str.append(str1)
    
df['utterances_str'] = utterances_str
#df.head(20)

In [58]:
data = df[['group', 'utterances_str']].rename(columns={"group":"label", "utterances_str":"text"})

data['label'] = '__label__' + data['label'].astype(str)

data.iloc[0:int(len(data)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [60]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)

classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=10)

2019-04-16 23:09:52,687 Reading data from .
2019-04-16 23:09:52,688 Train: train.csv
2019-04-16 23:09:52,689 Dev: dev.csv
2019-04-16 23:09:52,690 Test: test.csv
2019-04-16 23:09:53,269 this function is deprecated, use smart_open.open instead
2019-04-16 23:09:55,032 ----------------------------------------------------------------------------------------------------
2019-04-16 23:09:55,034 Evaluation method: MICRO_F1_SCORE
2019-04-16 23:09:55,035 ----------------------------------------------------------------------------------------------------


/home/gsus/anaconda3/envs/nlp_course/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  if __name__ == '__main__':


2019-04-16 23:09:56,368 epoch 1 - iter 0/177 - loss 0.02158587
2019-04-16 23:10:11,759 ----------------------------------------------------------------------------------------------------
2019-04-16 23:10:11,761 Exiting from training early.
2019-04-16 23:10:11,762 Saving model ...
2019-04-16 23:10:15,590 Done.
2019-04-16 23:10:15,593 ----------------------------------------------------------------------------------------------------
2019-04-16 23:10:15,597 Testing using best model ...
2019-04-16 23:10:15,600 loading file best-model.pt


/home/gsus/anaconda3/envs/nlp_course/lib/python3.6/site-packages/torch/serialization.py:542: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


KeyboardInterrupt: 

### Utterances por frase con Spacy

In [10]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
#path_cha = './Corpus/Pitt/Control/cookie/*.cha'
path_cha = './Corpus/Pitt/Dementia/cookie/*.cha'

processed = []
def process_with_symbols(path_cha):
    import glob
    import re
    
    files = sorted(glob.glob(path_cha))
    
    processed = []
    
    for file in files:
        document = open(file).read()
        doc = document.replace('\t', ' ').split('\n')
        
        
        pre_list = []
        for d in doc:
            match = re.findall(r'\*PAR:(.*)',d)
            if (len(match) != 0):
                pre_list.append(match)

        listt = []
        for p in pre_list:
            match = re.sub('x15.*?x15', '', str(p))
            match = re.sub('[\\\]', '', match)
            match = re.sub('["""]', '', match)
            match = re.sub('[\\\']', '', match)
            match = re.search(r'.*?\[(.*)].*', match).group(1)
            listt.append(match)
        
        processed.append(listt)
    
    return(processed)
    

utterances_symbol = process_with_symbols(path_cha)    
    
#utterances_symbol

In [43]:
# Deshacemos la lista embebida
#utterances_symbol_control = [item for sublist in utterances_symbol for item in sublist]
utterances_symbol_control = utterances_symbol
len(utterances_symbol_control)

243

In [45]:
# Deshacemos la lista embebida
#utterances_symbol_dementia = [item for sublist in utterances_symbol for item in sublist]
utterances_symbol_dementia = utterances_symbol
len(utterances_symbol_dementia)

309

In [46]:
group = list(['control']*len(utterances_symbol_control))
control_df = pd.DataFrame(
    {'label': group,
     'text': utterances_symbol_control
     })

In [47]:
group = list(['dementia']*len(utterances_symbol_dementia))
dementia_df = pd.DataFrame(
    {'label': group,
     'text': utterances_symbol_dementia
     })

In [ ]:
# Unimos los dos df , reorganizamos random y pasamos la colummna utterances de list -> str
df_spacy = pd.concat([control_df, dementia_df])
df_spacy = shuffle(df_spacy).reset_index(drop=True)
#df_spacy.head(20)

In [51]:
#data = df[['group', 'utterances_str']].rename(columns={"group":"label", "utterances_str":"text"})
data = df_spacy
data['label'] = '__label__' + data['label'].astype(str)

data.iloc[0:int(len(data)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [40]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)


trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=10)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
2019-04-16 19:48:32,384 Reading data from .
2019-04-16 19:48:32,385 Train: train.csv
2019-04-16 19:48:32,386 Dev: dev.csv
2019-04-16 19:48:32,387 Test: test.csv
2019-04-16 19:48:33,370 this function is deprecated, use smart_open.open instead
2019-04-16 19:48:34,702 ----------------------------------------------------------------------------------------------------
2019-04-16 19:48:34,703 Evaluation method: MICRO_F1_SCORE
2019-04-16 19:48:34,706 ----------------------------------------------------------------------------------------------------


/home/gsus/anaconda3/envs/nlp_course/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  # Remove the CWD from sys.path while we load stuff.


2019-04-16 19:48:35,225 epoch 1 - iter 0/175 - loss 0.02173432
2019-04-16 19:48:44,904 epoch 1 - iter 17/175 - loss 0.02209040
2019-04-16 19:48:54,585 epoch 1 - iter 34/175 - loss 0.02186388
2019-04-16 19:49:05,845 epoch 1 - iter 51/175 - loss 0.02178764
2019-04-16 19:49:17,049 epoch 1 - iter 68/175 - loss 0.02169157
2019-04-16 19:49:27,428 epoch 1 - iter 85/175 - loss 0.02149766
2019-04-16 19:49:38,461 epoch 1 - iter 102/175 - loss 0.02149710
2019-04-16 19:49:49,386 epoch 1 - iter 119/175 - loss 0.02147310
2019-04-16 19:50:00,122 epoch 1 - iter 136/175 - loss 0.02140294
2019-04-16 19:50:10,759 epoch 1 - iter 153/175 - loss 0.02142385
2019-04-16 19:50:21,705 epoch 1 - iter 170/175 - loss 0.02139754
2019-04-16 19:50:23,879 ----------------------------------------------------------------------------------------------------
2019-04-16 19:50:23,879 EPOCH 1 done: loss 0.0215 - lr 0.1000 - bad epochs 0
2019-04-16 19:50:34,255 DEV  : loss 0.02073390 - f-score 0.5788 - acc 0.4073
2019-04-16 19

2019-04-16 19:53:40,352 ----------------------------------------------------------------------------------------------------
2019-04-16 19:53:40,498 epoch 8 - iter 0/175 - loss 0.02415928
2019-04-16 19:53:42,453 epoch 8 - iter 17/175 - loss 0.01998995
2019-04-16 19:53:44,485 epoch 8 - iter 34/175 - loss 0.01922567
2019-04-16 19:53:46,867 epoch 8 - iter 51/175 - loss 0.01919771
2019-04-16 19:53:49,056 epoch 8 - iter 68/175 - loss 0.01917967
2019-04-16 19:53:51,367 epoch 8 - iter 85/175 - loss 0.01917253
2019-04-16 19:53:53,730 epoch 8 - iter 102/175 - loss 0.01912282
2019-04-16 19:53:55,918 epoch 8 - iter 119/175 - loss 0.01911164
2019-04-16 19:53:58,134 epoch 8 - iter 136/175 - loss 0.01918013
2019-04-16 19:54:00,239 epoch 8 - iter 153/175 - loss 0.01924460
2019-04-16 19:54:02,411 epoch 8 - iter 170/175 - loss 0.01933487
2019-04-16 19:54:02,880 ----------------------------------------------------------------------------------------------------
2019-04-16 19:54:02,881 EPOCH 8 done: loss

/home/gsus/anaconda3/envs/nlp_course/lib/python3.6/site-packages/torch/serialization.py:542: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


2019-04-16 19:55:05,019 MICRO_AVG: acc 0.4798 - f1-score 0.6485
2019-04-16 19:55:05,020 MACRO_AVG: acc 0.4547 - f1-score 0.61625
2019-04-16 19:55:05,021 control    tp: 125 - fp: 44 - fn: 201 - tn: 327 - precision: 0.7396 - recall: 0.3834 - accuracy: 0.3378 - f1-score: 0.5050
2019-04-16 19:55:05,021 dementia   tp: 327 - fp: 201 - fn: 44 - tn: 125 - precision: 0.6193 - recall: 0.8814 - accuracy: 0.5717 - f1-score: 0.7275
2019-04-16 19:55:05,021 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6485,
 'dev_score_history': [0.5788,
  0.5372,
  0.6132,
  0.6089,
  0.5559,
  0.5444,
  0.6132,
  0.6017,
  0.5946,
  0.6375],
 'train_loss_history': [0.021479301571503944,
  0.02083736562959775,
  0.020615555827039556,
  0.020280838996129192,
  0.02003447825845379,
  0.020013122574841787,
  0.01978355974301033,
  0.019387096359021694,
  0.019407997151016335,
  0.019220238228140122],
 'dev_loss_history': [0.020733904093503952,
  0.024556757882237434,
  0.020415527746081352,
  0.020108630880713463,
  0.023412024602293968,
  0.023150039836764336,
  0.02124148979783058,
  0.021458886563777924,
  0.021336788311600685,
  0.019638612866401672]}

In [41]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)


trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=20)

2019-04-16 20:56:03,929 Reading data from .
2019-04-16 20:56:03,931 Train: train.csv
2019-04-16 20:56:03,932 Dev: dev.csv
2019-04-16 20:56:03,933 Test: test.csv
2019-04-16 20:56:04,668 this function is deprecated, use smart_open.open instead
2019-04-16 20:56:06,059 ----------------------------------------------------------------------------------------------------
2019-04-16 20:56:06,060 Evaluation method: MICRO_F1_SCORE
2019-04-16 20:56:06,061 ----------------------------------------------------------------------------------------------------


/home/gsus/anaconda3/envs/nlp_course/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  # Remove the CWD from sys.path while we load stuff.


2019-04-16 20:56:06,788 epoch 1 - iter 0/175 - loss 0.02253088
2019-04-16 20:56:17,509 epoch 1 - iter 17/175 - loss 0.02199896
2019-04-16 20:56:28,634 epoch 1 - iter 34/175 - loss 0.02201932
2019-04-16 20:56:40,735 epoch 1 - iter 51/175 - loss 0.02184712
2019-04-16 20:56:54,415 epoch 1 - iter 68/175 - loss 0.02184557
2019-04-16 20:57:06,901 epoch 1 - iter 85/175 - loss 0.02177826
2019-04-16 20:57:18,612 epoch 1 - iter 102/175 - loss 0.02165912
2019-04-16 20:57:30,091 epoch 1 - iter 119/175 - loss 0.02157411
2019-04-16 20:57:41,496 epoch 1 - iter 136/175 - loss 0.02152725
2019-04-16 20:57:53,007 epoch 1 - iter 153/175 - loss 0.02147225
2019-04-16 20:58:04,720 epoch 1 - iter 170/175 - loss 0.02146622
2019-04-16 20:58:07,403 ----------------------------------------------------------------------------------------------------
2019-04-16 20:58:07,404 EPOCH 1 done: loss 0.0215 - lr 0.1000 - bad epochs 0
2019-04-16 20:58:19,850 DEV  : loss 0.02107741 - f-score 0.6017 - acc 0.4303
2019-04-16 20

2019-04-16 21:01:21,833 ----------------------------------------------------------------------------------------------------
2019-04-16 21:01:21,962 epoch 8 - iter 0/175 - loss 0.02239427
2019-04-16 21:01:24,002 epoch 8 - iter 17/175 - loss 0.01985807
2019-04-16 21:01:26,127 epoch 8 - iter 34/175 - loss 0.01943525
2019-04-16 21:01:28,393 epoch 8 - iter 51/175 - loss 0.01911848
2019-04-16 21:01:30,698 epoch 8 - iter 68/175 - loss 0.01926302
2019-04-16 21:01:32,869 epoch 8 - iter 85/175 - loss 0.01920509
2019-04-16 21:01:35,003 epoch 8 - iter 102/175 - loss 0.01928262
2019-04-16 21:01:37,298 epoch 8 - iter 119/175 - loss 0.01941797
2019-04-16 21:01:39,531 epoch 8 - iter 136/175 - loss 0.01956648
2019-04-16 21:01:41,838 epoch 8 - iter 153/175 - loss 0.01944977
2019-04-16 21:01:44,120 epoch 8 - iter 170/175 - loss 0.01946906
2019-04-16 21:01:44,576 ----------------------------------------------------------------------------------------------------
2019-04-16 21:01:44,579 EPOCH 8 done: loss

2019-04-16 21:04:30,371 DEV  : loss 0.01947201 - f-score 0.6562 - acc 0.4883
2019-04-16 21:04:30,905 TEST : loss 0.01964689 - f-score 0.6643 - acc 0.4973
2019-04-16 21:04:34,600 ----------------------------------------------------------------------------------------------------
2019-04-16 21:04:34,756 epoch 15 - iter 0/175 - loss 0.01722806
2019-04-16 21:04:36,813 epoch 15 - iter 17/175 - loss 0.01786978
2019-04-16 21:04:38,973 epoch 15 - iter 34/175 - loss 0.01844243
2019-04-16 21:04:41,321 epoch 15 - iter 51/175 - loss 0.01886955
2019-04-16 21:04:43,632 epoch 15 - iter 68/175 - loss 0.01870378
2019-04-16 21:04:45,816 epoch 15 - iter 85/175 - loss 0.01880593
2019-04-16 21:04:48,035 epoch 15 - iter 102/175 - loss 0.01889807
2019-04-16 21:04:50,558 epoch 15 - iter 119/175 - loss 0.01890585
2019-04-16 21:04:52,701 epoch 15 - iter 136/175 - loss 0.01886367
2019-04-16 21:04:54,816 epoch 15 - iter 153/175 - loss 0.01891219
2019-04-16 21:04:56,951 epoch 15 - iter 170/175 - loss 0.01884147
20

/home/gsus/anaconda3/envs/nlp_course/lib/python3.6/site-packages/torch/serialization.py:542: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


2019-04-16 21:07:24,615 MICRO_AVG: acc 0.4941 - f1-score 0.6614
2019-04-16 21:07:24,616 MACRO_AVG: acc 0.4729 - f1-score 0.6351
2019-04-16 21:07:24,617 control    tp: 137 - fp: 47 - fn: 189 - tn: 324 - precision: 0.7446 - recall: 0.4202 - accuracy: 0.3673 - f1-score: 0.5372
2019-04-16 21:07:24,617 dementia   tp: 324 - fp: 189 - fn: 47 - tn: 137 - precision: 0.6316 - recall: 0.8733 - accuracy: 0.5786 - f1-score: 0.7330
2019-04-16 21:07:24,618 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6614,
 'dev_score_history': [0.6017,
  0.5716,
  0.5702,
  0.6189,
  0.6175,
  0.6218,
  0.6562,
  0.6461,
  0.6461,
  0.6748,
  0.6762,
  0.6103,
  0.5688,
  0.6562,
  0.5874,
  0.6648,
  0.6017,
  0.5745,
  0.6748,
  0.6433],
 'train_loss_history': [0.021528136054617775,
  0.020918723131654254,
  0.020671948106422314,
  0.02021780730875869,
  0.020267455465824397,
  0.019829929500305158,
  0.019733682093096942,
  0.01954431365774557,
  0.019386975915095042,
  0.019476833241528043,
  0.019304383225429007,
  0.01926900947217962,
  0.019165536268497985,
  0.01903738566758472,
  0.018929886339042586,
  0.018756284930314362,
  0.01872031389063537,
  0.018818035534712643,
  0.018686705716542226,
  0.01857208439724859],
 'dev_loss_history': [0.021077409386634827,
  0.020837094634771347,
  0.021615762263536453,
  0.02029326930642128,
  0.020294327288866043,
  0.020499547943472862,
  0.01968453638255596,
  0.019614726305007935,
  0.01935340091586113,
  0.018986878916621208,
 

### Utterances por dialogo con Spacy

In [52]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path

corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)


trainer = ModelTrainer(classifier, corpus)
trainer.train('./', max_epochs=20)

2019-04-16 22:21:11,892 Reading data from .
2019-04-16 22:21:11,893 Train: train.csv
2019-04-16 22:21:11,895 Dev: dev.csv
2019-04-16 22:21:11,895 Test: test.csv
2019-04-16 22:21:12,685 this function is deprecated, use smart_open.open instead
2019-04-16 22:21:14,086 ----------------------------------------------------------------------------------------------------
2019-04-16 22:21:14,086 Evaluation method: MICRO_F1_SCORE
2019-04-16 22:21:14,088 ----------------------------------------------------------------------------------------------------


/home/gsus/anaconda3/envs/nlp_course/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  # Remove the CWD from sys.path while we load stuff.


2019-04-16 22:21:23,810 epoch 1 - iter 0/14 - loss 0.02370303
2019-04-16 22:21:37,920 epoch 1 - iter 1/14 - loss 0.02253457
2019-04-16 22:21:56,927 epoch 1 - iter 2/14 - loss 0.02221656
2019-04-16 22:22:12,869 epoch 1 - iter 3/14 - loss 0.02214558
2019-04-16 22:22:27,988 epoch 1 - iter 4/14 - loss 0.02189174
2019-04-16 22:22:38,407 epoch 1 - iter 5/14 - loss 0.02189414
2019-04-16 22:22:53,815 epoch 1 - iter 6/14 - loss 0.02172773
2019-04-16 22:23:09,900 epoch 1 - iter 7/14 - loss 0.02159470
2019-04-16 22:23:24,051 epoch 1 - iter 8/14 - loss 0.02207519
2019-04-16 22:23:41,541 epoch 1 - iter 9/14 - loss 0.02242262
2019-04-16 22:23:58,224 epoch 1 - iter 10/14 - loss 0.02200032
2019-04-16 22:24:12,067 epoch 1 - iter 11/14 - loss 0.02223844
2019-04-16 22:24:24,743 epoch 1 - iter 12/14 - loss 0.02216936
2019-04-16 22:24:35,611 epoch 1 - iter 13/14 - loss 0.02241316
2019-04-16 22:24:35,630 ----------------------------------------------------------------------------------------------------
201

2019-04-16 22:33:17,454 epoch 7 - iter 2/14 - loss 0.02137995
2019-04-16 22:33:26,023 epoch 7 - iter 3/14 - loss 0.02090790
2019-04-16 22:33:32,639 epoch 7 - iter 4/14 - loss 0.02097820
2019-04-16 22:33:39,211 epoch 7 - iter 5/14 - loss 0.02082480
2019-04-16 22:33:46,779 epoch 7 - iter 6/14 - loss 0.02121409
2019-04-16 22:33:53,370 epoch 7 - iter 7/14 - loss 0.02139490
2019-04-16 22:34:00,411 epoch 7 - iter 8/14 - loss 0.02104731
2019-04-16 22:34:06,011 epoch 7 - iter 9/14 - loss 0.02136488
2019-04-16 22:34:12,279 epoch 7 - iter 10/14 - loss 0.02133534
2019-04-16 22:34:18,007 epoch 7 - iter 11/14 - loss 0.02129730
2019-04-16 22:34:24,936 epoch 7 - iter 12/14 - loss 0.02126488
2019-04-16 22:34:30,063 epoch 7 - iter 13/14 - loss 0.02153050
2019-04-16 22:34:30,077 ----------------------------------------------------------------------------------------------------
2019-04-16 22:34:30,080 EPOCH 7 done: loss 0.0215 - lr 0.1000 - bad epochs 2
2019-04-16 22:34:31,240 DEV  : loss 0.02444947 - f

2019-04-16 22:42:21,667 epoch 13 - iter 2/14 - loss 0.02046368
2019-04-16 22:42:27,405 epoch 13 - iter 3/14 - loss 0.02084701
2019-04-16 22:42:34,966 epoch 13 - iter 4/14 - loss 0.02060765
2019-04-16 22:42:42,526 epoch 13 - iter 5/14 - loss 0.02094290
2019-04-16 22:42:49,851 epoch 13 - iter 6/14 - loss 0.02097185
2019-04-16 22:42:53,940 epoch 13 - iter 7/14 - loss 0.02094358
2019-04-16 22:43:00,285 epoch 13 - iter 8/14 - loss 0.02077139
2019-04-16 22:43:06,914 epoch 13 - iter 9/14 - loss 0.02070013
2019-04-16 22:43:13,834 epoch 13 - iter 10/14 - loss 0.02062144
2019-04-16 22:43:20,072 epoch 13 - iter 11/14 - loss 0.02070774
2019-04-16 22:43:26,741 epoch 13 - iter 12/14 - loss 0.02052631
2019-04-16 22:43:30,910 epoch 13 - iter 13/14 - loss 0.02076079
2019-04-16 22:43:30,923 ----------------------------------------------------------------------------------------------------
2019-04-16 22:43:30,925 EPOCH 13 done: loss 0.0208 - lr 0.0500 - bad epochs 0
2019-04-16 22:43:32,042 DEV  : loss 0

2019-04-16 22:51:27,371 epoch 19 - iter 2/14 - loss 0.02040735
2019-04-16 22:51:34,451 epoch 19 - iter 3/14 - loss 0.02039956
2019-04-16 22:51:41,538 epoch 19 - iter 4/14 - loss 0.02057913
2019-04-16 22:51:48,158 epoch 19 - iter 5/14 - loss 0.02043684
2019-04-16 22:51:53,768 epoch 19 - iter 6/14 - loss 0.02016216
2019-04-16 22:52:01,149 epoch 19 - iter 7/14 - loss 0.02037770
2019-04-16 22:52:06,696 epoch 19 - iter 8/14 - loss 0.02050842
2019-04-16 22:52:12,289 epoch 19 - iter 9/14 - loss 0.02044515
2019-04-16 22:52:17,241 epoch 19 - iter 10/14 - loss 0.02046256
2019-04-16 22:52:23,782 epoch 19 - iter 11/14 - loss 0.02048911
2019-04-16 22:52:30,568 epoch 19 - iter 12/14 - loss 0.02045103
2019-04-16 22:52:34,394 epoch 19 - iter 13/14 - loss 0.02073349
2019-04-16 22:52:34,410 ----------------------------------------------------------------------------------------------------
2019-04-16 22:52:34,411 EPOCH 19 done: loss 0.0207 - lr 0.0500 - bad epochs 0
2019-04-16 22:52:35,534 DEV  : loss 0

/home/gsus/anaconda3/envs/nlp_course/lib/python3.6/site-packages/torch/serialization.py:542: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


2019-04-16 22:54:15,773 MICRO_AVG: acc 0.5068 - f1-score 0.6727
2019-04-16 22:54:15,774 MACRO_AVG: acc 0.5023 - f1-score 0.66735
2019-04-16 22:54:15,776 control    tp: 15 - fp: 3 - fn: 15 - tn: 22 - precision: 0.8333 - recall: 0.5000 - accuracy: 0.4545 - f1-score: 0.6250
2019-04-16 22:54:15,779 dementia   tp: 22 - fp: 15 - fn: 3 - tn: 15 - precision: 0.5946 - recall: 0.8800 - accuracy: 0.5500 - f1-score: 0.7097
2019-04-16 22:54:15,781 ----------------------------------------------------------------------------------------------------


{'test_score': 0.6727,
 'dev_score_history': [0.5179,
  0.5,
  0.5,
  0.5,
  0.5893,
  0.5714,
  0.5357,
  0.5893,
  0.5357,
  0.6071,
  0.5,
  0.625,
  0.5,
  0.5893,
  0.6071,
  0.5536,
  0.6071,
  0.5179,
  0.5,
  0.5714],
 'train_loss_history': [0.0224131595791062,
  0.02195466889275445,
  0.021620179520172325,
  0.021275544788291394,
  0.02179550921835867,
  0.0219363022823723,
  0.02153050129105445,
  0.021024236603388712,
  0.021493510729601595,
  0.0215582278580352,
  0.02163666094782131,
  0.021051625275557814,
  0.02076079029074602,
  0.020783218396764225,
  0.02050525747459221,
  0.020830724109597756,
  0.020545900814116946,
  0.02040423760338435,
  0.020733493525965683,
  0.020353614878492292],
 'dev_loss_history': [0.02477421425282955,
  0.02522546984255314,
  0.02656552381813526,
  0.02986636757850647,
  0.024622010067105293,
  0.02458987571299076,
  0.02444946952164173,
  0.024392906576395035,
  0.024856051430106163,
  0.02417687512934208,
  0.025377456098794937,
  0.024